# Account & Position Discovery (TT-27 Epic)

Interactive exploration of the TastyTrade Account API — accounts, positions, balances, and transactions.
Used for discovery across all stories in the Account Management & Position Tracking epic.

In [ ]:
import json

import pandas as pd
from IPython.display import Markdown, display

from tastytrade.accounts import AccountsClient, Position
from tastytrade.config import RedisConfigManager
from tastytrade.connections import Credentials
from tastytrade.connections.requests import AsyncSessionHandler

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

# Connect

In [ ]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize(force=True)

credentials = Credentials(config=config, env="Live")
session = await AsyncSessionHandler.create(credentials)
client = AccountsClient(session)

print(f"Connected to {credentials.base_url}")

# Accounts

In [ ]:
accounts = await client.get_accounts()

accounts_data = []
for a in accounts:
    accounts_data.append({
        "Account": a.account_number,
        "Type": a.account_type_name,
        "Margin/Cash": a.margin_or_cash,
        "Futures": a.is_futures_approved,
        "Objective": a.investment_objective,
        "Options Level": a.suitable_options_level,
        "Opened": a.opened_at,
        "Funded": a.funding_date,
    })

df_accounts = pd.DataFrame(accounts_data)
display(Markdown(f"**{len(accounts)} accounts found**"))
display(df_accounts)

# Balances

In [ ]:
balances = []
for a in accounts:
    try:
        bal = await client.get_balances(a.account_number)
        balances.append({
            "Account": bal.account_number,
            "Cash": bal.cash_balance,
            "Net Liq": bal.net_liquidating_value,
            "Margin Equity": bal.margin_equity,
            "Equity BP": bal.equity_buying_power,
            "Derivative BP": bal.derivative_buying_power,
            "Day Trading BP": bal.day_trading_buying_power,
            "Maint Req": bal.maintenance_requirement,
            "Maint Excess": bal.maintenance_excess,
            "Long Equity": bal.long_equity_value,
            "Long Derivative": bal.long_derivative_value,
            "Long Futures": bal.long_futures_value,
            "Updated": bal.updated_at,
        })
    except Exception as e:
        balances.append({"Account": a.account_number, "Cash": f"Error: {e}"})

df_balances = pd.DataFrame(balances)
display(Markdown("**Account Balances**"))
display(df_balances)

# Positions

In [ ]:
all_positions: list[Position] = []
for a in accounts:
    positions = await client.get_positions(a.account_number)
    all_positions.extend(positions)
    print(f"{a.account_number} ({a.account_type_name}): {len(positions)} positions")

if all_positions:
    positions_data = []
    for p in all_positions:
        positions_data.append({
            "Account": p.account_number,
            "Symbol": p.symbol,
            "Underlying": p.underlying_symbol,
            "Type": p.instrument_type.value,
            "Qty": p.quantity,
            "Direction": p.quantity_direction.value,
            "Avg Open": p.average_open_price,
            "Mark": p.mark_price or p.mark,
            "Close": p.close_price,
            "Multiplier": p.multiplier,
            "Streamer": p.streamer_symbol,
            "Expires": p.expires_at,
            "Updated": p.updated_at,
        })

    df_positions = pd.DataFrame(positions_data)
    display(Markdown(f"**{len(all_positions)} total positions across {len(accounts)} accounts**"))
    display(df_positions)
else:
    display(Markdown("*No positions found*"))

## Positions by Instrument Type

In [ ]:
if all_positions:
    type_summary = df_positions.groupby("Type").agg(
        Count=("Symbol", "count"),
        Symbols=("Symbol", lambda x: ", ".join(sorted(set(x)))),
    ).sort_values("Count", ascending=False)
    display(Markdown("**Position breakdown by instrument type**"))
    display(type_summary)

    # Streamer symbol coverage
    has_streamer = sum(1 for p in all_positions if p.streamer_symbol)
    display(Markdown(f"**Streamer symbol coverage:** {has_streamer}/{len(all_positions)} positions have DXLink streamer symbols"))

# Transactions (Recent)

In [ ]:
# Transactions use raw API — not yet modeled in the SDK
acct = accounts[0].account_number if accounts else None

if acct:
    async with session.session.get(
        f"{session.base_url}/accounts/{acct}/transactions",
        params={"per-page": 20}
    ) as resp:
        data = await resp.json()
        txns = data.get("data", {}).get("items", [])

    if txns:
        txn_data = []
        for t in txns:
            txn_data.append({
                "Date": t.get("executed-at", t.get("transaction-date", "")),
                "Action": t.get("action", ""),
                "Symbol": t.get("underlying-symbol", t.get("symbol", "")),
                "Instrument": t.get("instrument-type", ""),
                "Description": t.get("description", ""),
                "Value": t.get("value", ""),
                "Effect": t.get("value-effect", ""),
                "Commission": t.get("commission", ""),
                "Fees": t.get("clearing-fees", ""),
            })
        df_txns = pd.DataFrame(txn_data)
        display(Markdown(f"**Last {len(txns)} transactions for {acct}**"))
        display(df_txns)
    else:
        display(Markdown("*No transactions found*"))

# Trading Status

In [ ]:
for a in accounts:
    async with session.session.get(
        f"{session.base_url}/accounts/{a.account_number}/trading-status"
    ) as resp:
        data = await resp.json()
        status = data.get("data", {})

    display(Markdown(f"### {a.account_number} ({a.account_type_name})"))
    status_items = [
        ("Day Trade Count", status.get("day-trade-count", "N/A")),
        ("Equities Margin Calc", status.get("equities-margin-calculation-type", "N/A")),
        ("Options Level", status.get("options-level", "N/A")),
        ("Short Calls", status.get("short-calls-enabled", "N/A")),
        ("Naked Options", status.get("are-naked-options-enabled", "N/A")),
        ("Futures Enabled", status.get("are-futures-enabled", "N/A")),
        ("Crypto Enabled", status.get("is-cryptocurrency-enabled", "N/A")),
        ("Max Order Size", status.get("max-symbol-quantity", "N/A")),
    ]
    df_status = pd.DataFrame(status_items, columns=["Setting", "Value"])
    display(df_status)

# API Response Explorer

Raw API inspection — use this to discover new fields and response shapes.

In [ ]:
# Change the endpoint to explore any API path
acct = accounts[0].account_number if accounts else "REPLACE_ME"
endpoint = f"/accounts/{acct}/positions"

async with session.session.get(f"{session.base_url}{endpoint}") as resp:
    data = await resp.json()
    print(json.dumps(data, indent=2, default=str)[:5000])

# Cleanup

In [ ]:
await session.close()
print("Session closed.")